In [1]:
## Pyspark Project

In [2]:
# helps to show where the pyspark is installed
import findspark
findspark.init()
findspark.find()

'/opt/anaconda3/envs/pyspark-env/lib/python3.12/site-packages/pyspark'

In [3]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
#from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import date_format

#build sparke session
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/23 15:23:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
print(spark.version)

3.4.1


In [5]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [6]:
# spark is an existing SparkSession
df1= spark.read.csv("datasets_sales.csv", header='true', inferSchema='true')

# Displays the content of the DataFrame to stdout
df1.show()

+--------+----------+--------+---------+-----------------+----------------+-------------------+--------------------+-----------+----------------+
|ID_ORDER|ID_PRODUCT|ID_BUYER|ID_SELLER|ID_SELLER_COUNTRY|ID_BUYER_COUNTRY|       DATE_PAYMENT|               BRAND|   CATEGORY|         REVENUE|
+--------+----------+--------+---------+-----------------+----------------+-------------------+--------------------+-----------+----------------+
|34033734|  13681706| 9666775|  6723223|              203|             170|2021-01-07 00:09:24|           Dsquared2|   clothing|        57.27456|
|33933770|  13575555| 8259842|  9983902|              222|             222|2021-01-05 00:23:10|               Eytys|      shoes|     168.7398624|
|33415990|  10239167| 8541994|   144764|              141|             204|2021-01-06 00:06:50|                Dior|      shoes|         126.414|
|33415990|  10239167| 8941774|   144764|              141|             204|2021-01-06 00:06:50|                Dior|      sh

In [7]:
#df1.select("category").show()

In [8]:
#df1.filter(df1['revenue'] > 1000).show()

In [9]:
# spark is an existing SparkSession
df2 = spark.read.csv("datasets_country.csv", header='true', inferSchema='true')

# Displays the content of the DataFrame to stdout
df2.show()

+----------+------+-------------------+
|ID_COUNTRY|REGION|            COUNTRY|
+----------+------+-------------------+
|       123|  EMEA|          Lithuania|
|       117|  EMEA|             Latvia|
|        67|  EMEA|            Estonia|
|        84|  EMEA|             Greece|
|       175|  EMEA|            Romania|
|       170|  EMEA|             Poland|
|       189|  EMEA|           Slovakia|
|        97|  EMEA|            Hungary|
|        56|  EMEA|     Czech Republic|
|        26|  AMER|            Bolivia|
|        18|  APAC|         Bangladesh|
|        50|  APAC|       Cook Islands|
|       115|  APAC|         Kyrgyzstan|
|       179|  AMER|        Saint Lucia|
|       211|  APAC|            Tokelau|
|       149|  APAC|              Nepal|
|       181|  APAC|              Samoa|
|       213|  AMER|Trinidad and Tobago|
|        22|  AMER|             Belize|
|        54|  AMER|               Cuba|
+----------+------+-------------------+
only showing top 20 rows



In [10]:
dfResult = df1.join(df2, df1.ID_SELLER_COUNTRY == df2.ID_COUNTRY, how="inner")
dfResult.show()

+--------+----------+--------+---------+-----------------+----------------+-------------------+--------------------+-----------+----------------+----------+------+--------------+
|ID_ORDER|ID_PRODUCT|ID_BUYER|ID_SELLER|ID_SELLER_COUNTRY|ID_BUYER_COUNTRY|       DATE_PAYMENT|               BRAND|   CATEGORY|         REVENUE|ID_COUNTRY|REGION|       COUNTRY|
+--------+----------+--------+---------+-----------------+----------------+-------------------+--------------------+-----------+----------------+----------+------+--------------+
|34033734|  13681706| 9666775|  6723223|              203|             170|2021-01-07 00:09:24|           Dsquared2|   clothing|        57.27456|       203|  EMEA|        Sweden|
|33933770|  13575555| 8259842|  9983902|              222|             222|2021-01-05 00:23:10|               Eytys|      shoes|     168.7398624|       222|  EMEA|United Kingdom|
|33415990|  10239167| 8541994|   144764|              141|             204|2021-01-06 00:06:50|          

In [11]:
# Register the DataFrame as a temporary view
dfResult.createOrReplaceTempView("dfResult")
result = spark.sql("""SELECT ID_ORDER, ID_PRODUCT, ID_BUYER, ID_SELLER, DATE_PAYMENT, BRAND, CATEGORY, REVENUE, REGION, COUNTRY  FROM dfResult""")

In [12]:
result.show()

+--------+----------+--------+---------+-------------------+--------------------+-----------+----------------+------+--------------+
|ID_ORDER|ID_PRODUCT|ID_BUYER|ID_SELLER|       DATE_PAYMENT|               BRAND|   CATEGORY|         REVENUE|REGION|       COUNTRY|
+--------+----------+--------+---------+-------------------+--------------------+-----------+----------------+------+--------------+
|34033734|  13681706| 9666775|  6723223|2021-01-07 00:09:24|           Dsquared2|   clothing|        57.27456|  EMEA|        Sweden|
|33933770|  13575555| 8259842|  9983902|2021-01-05 00:23:10|               Eytys|      shoes|     168.7398624|  EMEA|United Kingdom|
|33415990|  10239167| 8541994|   144764|2021-01-06 00:06:50|                Dior|      shoes|         126.414|  EMEA|        Monaco|
|33415990|  10239167| 8941774|   144764|2021-01-06 00:06:50|                Dior|      shoes|         126.414|  EMEA|        Monaco|
|33415990|  10239167| 8541774|   944764|2021-01-06 00:06:50|         

In [13]:
result_with_date = result.withColumn("DATE_PAYMENT", date_format("DATE_PAYMENT", "yyyy-MM-dd"))
result_with_date.show()

+--------+----------+--------+---------+------------+--------------------+-----------+----------------+------+--------------+
|ID_ORDER|ID_PRODUCT|ID_BUYER|ID_SELLER|DATE_PAYMENT|               BRAND|   CATEGORY|         REVENUE|REGION|       COUNTRY|
+--------+----------+--------+---------+------------+--------------------+-----------+----------------+------+--------------+
|34033734|  13681706| 9666775|  6723223|  2021-01-07|           Dsquared2|   clothing|        57.27456|  EMEA|        Sweden|
|33933770|  13575555| 8259842|  9983902|  2021-01-05|               Eytys|      shoes|     168.7398624|  EMEA|United Kingdom|
|33415990|  10239167| 8541994|   144764|  2021-01-06|                Dior|      shoes|         126.414|  EMEA|        Monaco|
|33415990|  10239167| 8941774|   144764|  2021-01-06|                Dior|      shoes|         126.414|  EMEA|        Monaco|
|33415990|  10239167| 8541774|   944764|  2021-01-06|                Dior|      shoes|         126.414|  EMEA|        